<a href="https://colab.research.google.com/github/crunchdomo/llm_conversation/blob/main/test-base-Llama-3-8B-Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install 'accelerate>=0.26.0'
%pip install torch
%pip install transformers
%pip install accelerate
%pip install -U transformers bitsandbytes
!python -m pip install optimum


In [ ]:
print(f"GPU available: {torch.cuda.is_available()}")
print(f"Using device: {pipe.device}")


GPU available: True
Using device: cpu


In [8]:
import torch
from transformers import pipeline, AutoTokenizer
import json

device = 0 if torch.cuda.is_available() else -1  # 0 = first GPU, -1 = CPU


def format_llama3_prompt(messages):
    B_INST, E_INST = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>", "<|eot_id|>"
    return "".join(
        f"{B_INST}{msg['content']}{E_INST}" if msg['role'] == 'user'
        else f"<|start_header_id|>assistant<|end_header_id|>{msg['content']}<|eot_id|>"
        for msg in messages
    )

# Initialize the tokenizer separately
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

pipe = pipeline(
    task="text-generation",
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    max_new_tokens=512,
    device=device,
    torch_dtype=torch.bfloat16,  # Use bfloat16 for all devices
    # Pass the tokenizer to the pipeline function instead of model_kwargs
    tokenizer=tokenizer,
    # Remove 'chat_template' from model_kwargs as it's not a valid argument for LlamaForCausalLM
    # generation_config={"chat_template": format_llama3_prompt}  # You can try using generation_config instead if needed
    # If you need to use a custom formatting function, consider applying it to the prompt before passing it to the pipeline
)

pipe.model = pipe.model.to(torch.bfloat16)
pipe.tokenizer.pad_token = pipe.tokenizer.eos_token


def cook_recipe(recipe_data, automated_inputs):
    # Initialize fresh conversation
    chat = [
        {
            "role": "system",
            "content": """You are a master chef. When the user asks to begin, wait for them to confirm they are ready before starting the recipe. "
                          "Once the user says they are ready, guide them through the recipe step-by-step. For each step:\n"
                          "- Use metric units.\n"
                          "- Number each step.\n"
                          "- End each step with 'Awaiting confirmation.'\n"
                          "- Do not repeat previous steps or instructions.\n"
                          "- After the last step, say 'CONVO-COMPLETE'.
                      """
        }
    ]

    # Structured recipe submission
    chat.append({
        "role": "user",
        "content": f"""Guide me through this recipe step-by-step:

        TITLE: {recipe_data['title']}

        INGREDIENTS:
        {chr(10).join(recipe_data['ingredients'])}

        INSTRUCTIONS:
        {recipe_data['instructions']}"""
    })

    # Process automated inputs
    for user_input in automated_inputs:
        # Format the prompt for the model
        prompt = format_llama3_prompt(chat)
        outputs = pipe(prompt)
        chef_response = outputs[0]["generated_text"][len(prompt):].strip()  # Get only the new output

        print(f"\nChef: {chef_response}")
        chat.append({"role": "assistant", "content": chef_response})

        # Check for 'CONVO-COMPLETE' in chef's response (case-insensitive)
        if "convo-complete" in chef_response.lower():
            break

        print(f"\nTrainee: {user_input}")
        chat.append({"role": "user", "content": user_input})

        if "exit" in user_input.lower():
            break


    save_conversation(chat)
    return chat


def save_conversation(chat_history, filename="cooking_session.json"):
    with open(filename, 'w') as f:
        json.dump(chat_history, f, indent=2)
    print(f"Saved to {filename}")

# Recipe data structure
recipe = {
    "title": "Miso-Butter Roast Chicken With Acorn Squash Panzanella",
    "ingredients": [
        '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',
        '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',
        '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
        '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',
        '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
        '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
        '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced',
        '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour',
        '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine',
        '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper'
    ],
    "instructions": """
        1. Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.

        2. Meanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½""-thick wedges; arrange on a rimmed baking sheet.

        3. Combine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.

        4. Add bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.
        Place onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.

        5. Place a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.

        6. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.

        7. Using your fingers, mash flour and butter in a small bowl to combine.

        8. Set reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.

        9. Serve chicken with gravy and squash panzanella alongside.
            """
}

# Predefined automated trainee inputs
automated_inputs = [
    "Got it! Ready for step 1.",
    "Done Continue.",
    "Please Continue.",
    "Done Continue.",
    "Please Continue.",
    "Done Continue.",
    "Please Continue.",
    "Done Continue.",
    "Please Continue.",
    "Done Continue.",
    "Please Continue.",
    "exit"
]

# Start cooking session with automated inputs
cook_recipe(recipe, automated_inputs)


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Chef: assistant

**Step 7:**
Using your fingers, mash the ¼ cup all-purpose flour and 2 Tbsp. unsalted butter in a small bowl to combine.

Awaiting confirmation...

Trainee: Done Continue.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Chef: assistant

**Step 8:**
Set the reserved skillet with the chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add the ¼ cup dry white wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until the bits are loosened and the wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add the butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add the 2 cups unsalted chicken broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in the 2 tsp. white miso. Taste and season with salt and freshly ground black pepper.

Awaiting confirmation...

Trainee: Please Continue.

Chef: assistant

**Step 9:**
Serve the chicken with the gravy and squash panzanella alongside.

CONVO-COMPLETE
Saved to cooking_session.js

[{'role': 'system',
  'content': 'You are a master chef. When the user asks to begin, wait for them to confirm they are ready before starting the recipe. "\n                          "Once the user says they are ready, guide them through the recipe step-by-step. For each step:\n"\n                          "- Use metric units.\n"\n                          "- Number each step.\n"\n                          "- End each step with \'Awaiting confirmation.\'\n"\n                          "- Do not repeat previous steps or instructions.\n"\n                          "- After the last step, say \'CONVO-COMPLETE\'.\n                      '},
 {'role': 'user',
  'content': 'Guide me through this recipe step-by-step:\n\n        TITLE: Miso-Butter Roast Chicken With Acorn Squash Panzanella\n\n        INGREDIENTS:\n        1 (3½–4-lb.) whole chicken\n2¾ tsp. kosher salt, divided, plus more\n2 small acorn squash (about 3 lb. total)\n2 Tbsp. finely chopped sage\n1 Tbsp. finely chopped rosemary\n6 T